## Case study of using multi-index dataframe

### Methods
#### 1. DataFrame.loc - A general solution for selection by label (+ pd.IndexSlice for more complex applications involving slices)

#### 2. DataFrame.xs - Extract a particular cross section from a Series/DataFrame.

#### 3. DataFrame.query - Specify slicing and/or filtering operations dynamically (i.e., as an expression that is evaluated dynamically. Is more applicable to some scenarios than others. Also see this section of the docs for querying on MultiIndexes.

#### 4. Boolean indexing with a mask generated using MultiIndex.get_level_values (often in conjunction with Index.isin, especially when filtering with multiple values). This is also quite useful in some circumstances.

In [4]:
import pandas as pd
import numpy as np
mux = pd.MultiIndex.from_arrays([
    list('aaaabbbbbccddddd'),
    list('tuvwtuvwtuvwtuvw')
], names=['one', 'two'])

df = pd.DataFrame({'col': np.arange(len(mux))}, mux)


### Q1 : Selecting a Single Item
How do I select rows having "a" in level "one"? Additionally, how would I be able to drop level "one" in the output?

In [5]:
df.loc[['a']]

col
one two     
a   t      0
    u      1
    v      2
    w      3

In [10]:
df.loc['a']

,col
two,
t,0
u,1
v,2
w,3


In [6]:
df.xs('a', level=0, axis=0, drop_level=False)

col
one two     
a   t      0
    u      1
    v      2
    w      3

In [12]:
df.xs('a', level=0, axis=0, drop_level=True)

,col
two,
t,0
u,1
v,2
w,3


In [7]:
df.query("one == 'a'")

col
one two     
a   t      0
    u      1
    v      2
    w      3

In [8]:
#using get_level_values
df[df.index.get_level_values('one') == 'a']

col
one two     
a   t      0
    u      1
    v      2
    w      3

In [13]:
v = df.loc[['a']]
print(v)

         col
one two     
a   t      0
    u      1
    v      2
    w      3


In [14]:
print(v.index)

MultiIndex(levels=[['a', 'b', 'c', 'd'], ['t', 'u', 'v', 'w']],
           codes=[[0, 0, 0, 0], [0, 1, 2, 3]],
           names=['one', 'two'])


In [15]:
#You can get rid of these levels using MultiIndex.remove_unused_levels:
v.index = v.index.remove_unused_levels()

print(v.index)

MultiIndex(levels=[['a'], ['t', 'u', 'v', 'w']],
           codes=[[0, 0, 0, 0], [0, 1, 2, 3]],
           names=['one', 'two'])


### Q1b :
How do I slice all rows with value "t" on level "two"?

In [16]:
df.loc[(slice(None), 't'), :]

col
one two     
a   t      0
b   t      4
    t      8
d   t     12

In [18]:
#using the pd.IndexSlice API here
idx = pd.IndexSlice
df.loc[idx[:, 't'], :]

col
one two     
a   t      0
b   t      4
    t      8
d   t     12

In [19]:
df.xs('t', axis=0, level=1, drop_level=False)

col
one two     
a   t      0
b   t      4
    t      8
d   t     12

In [20]:
df.query("two == 't'")

col
one two     
a   t      0
b   t      4
    t      8
d   t     12

In [21]:
df[df.index.get_level_values('two') == 't']

col
one two     
a   t      0
b   t      4
    t      8
d   t     12

### Q2 : Selecting Multiple Values in a Level
How can I select rows corresponding to items "b" and "d" in level "one"?

In [22]:
df.loc[['b', 'd']]

col
one two     
b   t      4
    u      5
    v      6
    w      7
    t      8
d   w     11
    t     12
    u     13
    v     14
    w     15

In [23]:
items = ['b', 'd']
df.query("one in @items")

col
one two     
b   t      4
    u      5
    v      6
    w      7
    t      8
d   w     11
    t     12
    u     13
    v     14
    w     15

In [24]:
df[df.index.get_level_values("one").isin(['b', 'd'])]

col
one two     
b   t      4
    u      5
    v      6
    w      7
    t      8
d   w     11
    t     12
    u     13
    v     14
    w     15

### Q2b:
How would I get all values corresponding to "t" and "w" in level "two"?

In [25]:
df.loc[pd.IndexSlice[:, ['t', 'w']], :] 

col
one two     
a   t      0
    w      3
b   t      4
    w      7
    t      8
d   w     11
    t     12
    w     15

In [26]:
items = ['t', 'w']
df.query("two in @items")

col
one two     
a   t      0
    w      3
b   t      4
    w      7
    t      8
d   w     11
    t     12
    w     15

In [27]:
df[df.index.get_level_values('two').isin(['t', 'w'])]

col
one two     
a   t      0
    w      3
b   t      4
    w      7
    t      8
d   w     11
    t     12
    w     15

### Q3: Slicing a Single Cross Section (x, y)
How do I retrieve a cross section, i.e., a single row having a specific values for the index from df? Specifically, how do I retrieve the cross section of ('c', 'u')

In [28]:
df.loc[('c', 'u'), :]

C:\Users\Owner\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


,,col
one,two,
c,u,9


In [29]:
df.loc[pd.IndexSlice[('c', 'u')]]

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,,col
one,two,
c,u,9


In [30]:
df_sort = df.sort_index()
df_sort.loc[('c', 'u')]

,,col
one,two,
c,u,9


In [31]:
df.xs(('c', 'u'))

C:\Users\Owner\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2874: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


,,col
one,two,
c,u,9


In [32]:
df.query("one == 'c' and two == 'u'")

,,col
one,two,
c,u,9


In [33]:
m1 = (df.index.get_level_values('one') == 'c')
m2 = (df.index.get_level_values('two') == 'u')
df[m1 & m2]

,,col
one,two,
c,u,9


### Q4: Slicing Multiple Cross Sections [(a, b), (c, d), ...]
How do I select the two rows corresponding to ('c', 'u'), and ('a', 'w')?

In [34]:
df.loc[[('c', 'u'), ('a', 'w')]]

,,col
one,two,
c,u,9
a,w,3


In [35]:
#TOO COMPLICATED!!!
cses = [('c', 'u'), ('a', 'w')]
levels = ['one', 'two']
# This is a useful check to make in advance.
assert all(len(levels) == len(cs) for cs in cses) 

query = '(' + ') or ('.join([
    ' and '.join([f"({l} == {repr(c)})" for l, c in zip(levels, cs)]) 
    for cs in cses
]) + ')'

print(query)
# ((one == 'c') and (two == 'u')) or ((one == 'a') and (two == 'w'))

df.query(query)

((one == 'c') and (two == 'u')) or ((one == 'a') and (two == 'w'))


,,col
one,two,
a,w,3
c,u,9


### Q5: One Item Sliced per Level
How can I retrieve all rows corresponding to "a" in level "one" or "t" in level "two"?

In [36]:
pd.concat([
    df.loc[['a'],:], df.loc[pd.IndexSlice[:, 't'],:]
])

col
one two     
a   t      0
    u      1
    v      2
    w      3
    t      0
b   t      4
    t      8
d   t     12

In [37]:
v = pd.concat([
        df.loc[['a'],:], df.loc[pd.IndexSlice[:, 't'],:]
])
v[~v.index.duplicated()]

col
one two     
a   t      0
    u      1
    v      2
    w      3
b   t      4
d   t     12

In [38]:
df.query("one == 'a' or two == 't'")

col
one two     
a   t      0
    u      1
    v      2
    w      3
b   t      4
    t      8
d   t     12

In [39]:
m1 = (df.index.get_level_values('one') == 'c')
m2 = (df.index.get_level_values('two') == 'u')
df[m1 | m2]

col
one two     
a   u      1
b   u      5
c   u      9
    v     10
d   u     13

### Q6: Arbitrary Slicing
How can I slice specific cross sections? For "a" and "b", I would like to select all rows with sub-levels "u" and "v", and for "d", I would like to select rows with sub-level "w".

In [40]:
keys = [('a', 'u'), ('a', 'v'), ('b', 'u'), ('b', 'v'), ('d', 'w')]
df.loc[keys, :]

col
one two     
a   u      1
    v      2
b   u      5
    v      6
d   w     11
    w     15

In [41]:
pd.concat([
     df.loc[(('a', 'b'), ('u', 'v')), :], 
     df.loc[('d', 'w'), :]
   ], axis=0)

col
one two     
a   u      1
    v      2
b   u      5
    v      6
d   w     11
    w     15

### Q7: Inequality-based filtering on Numeric Levels
How do I get all rows where values in level "two" are greater than 5?

In [45]:
np.random.seed(0)
mux2 = pd.MultiIndex.from_arrays([
    list('aaaabbbbbccddddd'),
    np.random.choice(10, size=16)
], names=['one', 'two'])

df2 = pd.DataFrame({'col': np.arange(len(mux2))}, mux2)

In [46]:
df2.query("two > 5")

col
one two     
b   7      4
    9      5
c   7     10
d   6     11
    8     12
    8     13
    6     15

In [47]:
df2[df2.index.get_level_values('two') > 5]

col
one two     
b   7      4
    9      5
c   7     10
d   6     11
    8     12
    8     13
    6     15

In [ ]:
#https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe